# Prioritizing Risk Factors of Heart Failure from UK Biobank Using Explainable Artificial Intelligence

## Authors
Arkarachai Fungtammasan, Chiao-Feng Lin, Daniel Quang, Yih-Chii Hwang, Jason Chin

## Abstract
Predicting diseases using genetic variation, phenotypic traits, or environmental factors is one of the most important goals for precision health. With the recent advancement in machine learning techniques and the availability of large-scale genotype-phenotype databases, sophisticated machine learning methods have become widely adopted by the scientific community for predicting diseases. Many of these techniques offer more flexibility in terms of model assumption compared to traditional statistical methods and the ability to utilize a large amount of data as they have been developed to handle a larger amount of data in other scientific domains. However, the lack of interpretability and explainability of these models faces challenges in detecting errors and building trust. 

We demonstrate the use case of explainable artificial intelligence in predicting heart failure, one of the global major causes of death. We extracted the baseline data, blood measured, and reported lifestyle features from UK Biobank using UKB Research Analysis Platform. We experimented with the state-of-art black-box modeling tools such as LightGBM and XGBoost and estimated the contribution of each predictive feature using SHAP. In addition, we also used a recently developed glass-box machine learning model, Explainable Boosting Machine, which directly measures the impact of each predictor and their interactions instead of indirect inference. 

We found that sophisticated blackbox and glassbox machine learning methods yield similar predictive power. Age and sex are the most important features for most model classes regardless of algorithms, missing value handling strategies, or glassbox vs blackbox feature important analysis. The rest of the important features such as BMI, blood-measured C-reactive protein, cholesterol, Hemoglobin A1c are contributing to the minority of the population, but in large effect. Some features such as BMI also have non-linear contributions to model prediction. Our study exemplifies how machine learning could be used to study the impact of risk factors for individual patient. 



# Table of content
1. Load dataset in to instance with Spark 
2. Choose fields and put in Spark dataframe
3. Use readable field name and save as pandas pickle
4. Close Spark and setup data/library in ML_IP instance
5. Mark missing values
6. Process variables
7. Univariate analysis
8. Basic machine learning 
9. Interpretable and explainable ML




### Note that data exploration has not been included in this demo. Some data exploration could be done using UK Biobank cohort browser. 

# 1. Load dataset in to instance with Spark

### Launch Spark cluster HAIL-0.2.61 using instance mem1_ssd1_v2_x8

In [ ]:
import dxdata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats
import json
# Initialize dxdata engine
engine = dxdata.connect(dialect="hive+pyspark")
pt = engine.execute("SET spark.sql.shuffle.partitions=50").to_pandas()

In [ ]:
DATASET_ID = "project-G59Y0xQJ51ZZFXpfFpfxQZzj:record-G59f2VjJk7QPJpF04v0619QG" # app46926_20210928164649.dataset
dataset = dxdata.load_dataset(id=DATASET_ID)  


### Dataset "entities" are virtual tables linked to one another.

The main entity is "participant" and corresponds to most pheno fields. Additional entities correspond to linked health care data.
Entities starting with "hesin" are for hospital records; entities starting with "gp" are for GP records, etc.

In [ ]:
dataset.entities

### Accessing the main 'participant' entity

In [ ]:
participant = dataset["participant"]

In [ ]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

#### Selecting participant fields by field index, instance index, array index

For the main participant pheno entity, RAP uses field names with the following convention:

|Type of field|Syntax for field name|Example|
|:------------|---------------------|-------|
|Neither instanced nor arrayed|`p<FIELD>`|`p31`|
|Instanced but not arrayed|`p<FIELD>_i<INSTANCE>`|`p40005_i0`|
|Arrayed but not instanced|`p<FIELD>_a<ARRAY>`|`p41262_a0`|
|Instanced and arrayed|`p<FIELD>_i<INSTANCE>_a<ARRAY>`|`p93_i0_a0`|

If you know exactly the field names you want to work with, put them in a string array (we will see later how to use that):

#### Looking up fields by id

If you know the field id but you are not sure if it is instanced or arrayed, and want to grab all instances/arrays (if any), use this:

In [ ]:
def field_names_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted([field.name for field in fields], key=lambda n: LooseVersion(n))

#### Looking up fields by title

If you remember part of the field title, use this:

In [ ]:
def fields_by_title_keyword(keyword):
    import pandas as pd
    import re
    from distutils.version import LooseVersion
    regex = re.compile("^.*" + re.escape(keyword) + ".*$", re.I)
    fields = participant.find_fields(boolean_func=lambda f: getattr(f, "title") and bool(regex.match(f.title)))
    return pd.DataFrame(data=[(f.name, f.title, " > ".join(f.folder_path)) for f in sorted(fields, key=lambda n: LooseVersion(n.name))], columns=["name","title", "category"])

def field_names_by_title_keyword(keyword):
    return list(fields_by_title_keyword(keyword)["name"])

# 2. Choose fields and put in Spark dataframe

In [ ]:
feature_code_mapping={'BMI': 21001,
 'HDL': 30760,
 'LDL': 30780,                    
 'HbA1c': 30750,
 'ICD10': 41270,
 'ICD9': 41271,
 'accleration_average':90012,
 'accleration_good_calibrate':90016,
 'accleration_good_wear_time':90015,
 'TV': 1070,
 'age': 21022,
 'alcohol': 1558,
 'all_MET': 22040,
 'c_reactive_protein': 30710,
 'computer': 1080,
 'cooked_vegie': 1289,
 'education': 6138,
 'fresh_fruit': 1309,
 'job': 22601,
 'moderate_activity': 894,
 'process_meat': 1349,
 'raw_vegie': 1299,
 'sex': 31,
 'sleep': 1160,
 'smoke': 20116,
 'total_cholesteral': 30690,
 'triglycerides': 30870,
 'townsend': 189,
 'vigorous_activity': 914,
 'walk': 874,
 'day_walk': 864,
 'day_moderate': 884,
 'day_vigorous': 904,
 'ethnic_background':21000
}
#was trying to change smoke from 20116 to 22506, but revert back due to high missing value

In [ ]:
HF_paper_list=[
"age",
"sex",
"BMI",
"alcohol",
"sleep",
"walk",
"moderate_activity",
"vigorous_activity",
"TV",
"computer",
"smoke",
"accleration_average",
'accleration_good_calibrate',
'accleration_good_wear_time',
"townsend",
"total_cholesteral",
"ICD10",
"ICD9",
"day_vigorous",
"day_moderate",
"day_walk",
"ethnic_background",
"HDL",
"LDL",
"triglycerides",
"HbA1c",
"c_reactive_protein",
"cooked_vegie",
"raw_vegie",
"fresh_fruit",
"process_meat",
]

In [ ]:
feature_list=HF_paper_list

In [ ]:
field_names = [] 
for feature in feature_list:
    print(feature)
    print(field_names_for_id(feature_code_mapping[feature]))
    field_names+=field_names_for_id(feature_code_mapping[feature])

### Grabbing fields into a spark dataframe

In [ ]:
engine = dxdata.connect(dialect="hive+pyspark")
df = participant.retrieve_fields(engine=engine, names=field_names)

In [ ]:
ashg_df_pandas=df.toPandas()

# 3. Use readable field name and save as pandas pickle

In [ ]:
characters_to_remove = "[-\|\,]"
title_dict={}
for field_name in field_names:
    new_name=participant[field_name].title
    new_name = re.sub(characters_to_remove, "", new_name)
    new_name = new_name.replace(' ','_')
    new_name = new_name.replace('_/_','_')
    new_name = re.sub("__+", "_", new_name)
    title_dict[field_name]=new_name
title_dict

In [ ]:
%%time
ashg_df_pandas.rename(columns=title_dict,inplace=True)

### Save to pickle, so you don't have to use cluster

In [ ]:
ashg_df_pandas.describe()

In [ ]:
ashg_df_pandas.to_pickle("./ashg_df_pandas.pkl")

In [ ]:
! dx mkdir ASHGdemo
! dx upload ashg_df_pandas.pkl --path /ASHGdemo/ashg_df_pandas.pkl

### Check in the project if pandas pickle file have been completely uploaded

# 4. Close Spark and setup data/library in ML_IP instance

### Close Spark cluster and launch a Python_R instance with mem1_ssd1_v2_x8 instead

This is done using UKB-RAP UI.

### On new instance, install new version of Seaborn

In [ ]:
pip install seaborn==0.11.1

#### Now restart kernel to make new Seaborn in effect.

### Import required library

In [ ]:
from IPython.display import Image

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats
import json

In [ ]:
print(sns.__version__)

### Load pandas pickle

In [ ]:
! dx download ASHGdemo/ashg_df_pandas.pkl

In [ ]:
ashg_df_pandas=pd.read_pickle("./ashg_df_pandas.pkl")

In [ ]:
ashg_df_pandas.info()

# 5. Mark missing values

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
ashg_df_pandas.describe()

### Observations
1. There are both NaN and unexpected negative values. These values should be processed to have correct representation. 
2. Q1, Q2, Q3 shows that most of these features have discrete values. Some of them should be nominal and ordinal. 


In [ ]:
ashg_df_pandas.isna().sum()

In [ ]:
ashg_df_pandas.replace({
    'Alcohol_intake_frequency._Instance_0':{-3:np.nan},
    'Alcohol_intake_frequency._Instance_1':{-3:np.nan},
    'Alcohol_intake_frequency._Instance_2':{-3:np.nan},
    'Alcohol_intake_frequency._Instance_3':{-3:np.nan},
    'Overall_health_rating_Instance_0':{-3:np.nan,-1:np.nan},
    'Overall_health_rating_Instance_1':{-3:np.nan,-1:np.nan},
    'Overall_health_rating_Instance_2':{-3:np.nan,-1:np.nan},
    'Overall_health_rating_Instance_3':{-3:np.nan,-1:np.nan},
    'Smoking_status_Instance_0':{-3:np.nan},
    'Smoking_status_Instance_1':{-3:np.nan},
    'Smoking_status_Instance_2':{-3:np.nan},
    'Smoking_status_Instance_3':{-3:np.nan},
    'Sleep_duration_Instance_0':{-3:np.nan,-1:np.nan},
    'Sleep_duration_Instance_1':{-3:np.nan,-1:np.nan},
    'Sleep_duration_Instance_2':{-3:np.nan,-1:np.nan},
    'Sleep_duration_Instance_3':{-3:np.nan,-1:np.nan},
    'Duration_of_walks_Instance_0':{-3:np.nan,-1:np.nan},
    'Duration_of_walks_Instance_1':{-3:np.nan,-1:np.nan},
    'Duration_of_walks_Instance_2':{-3:np.nan,-1:np.nan},
    'Duration_of_walks_Instance_3':{-3:np.nan,-1:np.nan},
    'Duration_of_moderate_activity_Instance_0':{-3:np.nan,-1:np.nan},
    'Duration_of_moderate_activity_Instance_1':{-3:np.nan,-1:np.nan},
    'Duration_of_moderate_activity_Instance_2':{-3:np.nan,-1:np.nan},
    'Duration_of_moderate_activity_Instance_3':{-3:np.nan,-1:np.nan},
    'Duration_of_vigorous_activity_Instance_0':{-3:np.nan,-1:np.nan},
    'Duration_of_vigorous_activity_Instance_1':{-3:np.nan,-1:np.nan},
    'Duration_of_vigorous_activity_Instance_2':{-3:np.nan,-1:np.nan},
    'Duration_of_vigorous_activity_Instance_3':{-3:np.nan,-1:np.nan},
    'Time_spent_watching_television_(TV)_Instance_0':{-3:np.nan,-1:np.nan,-10:0},
    'Time_spent_watching_television_(TV)_Instance_1':{-3:np.nan,-1:np.nan,-10:0},
    'Time_spent_watching_television_(TV)_Instance_2':{-3:np.nan,-1:np.nan,-10:0},
    'Time_spent_watching_television_(TV)_Instance_3':{-3:np.nan,-1:np.nan,-10:0},
    'Time_spent_using_computer_Instance_0':{-3:np.nan,-1:np.nan,-10:0},
    'Time_spent_using_computer_Instance_1':{-3:np.nan,-1:np.nan,-10:0},
    'Time_spent_using_computer_Instance_2':{-3:np.nan,-1:np.nan,-10:0},
    'Time_spent_using_computer_Instance_3':{-3:np.nan,-1:np.nan,-10:0},
    'Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_0':{-3:np.nan,-1:np.nan},
    'Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_1':{-3:np.nan,-1:np.nan},
    'Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_2':{-3:np.nan,-1:np.nan},
    'Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_3':{-3:np.nan,-1:np.nan},
    'Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_0':{-3:np.nan,-1:np.nan},
    'Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_1':{-3:np.nan,-1:np.nan},
    'Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_2':{-3:np.nan,-1:np.nan},
    'Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_3':{-3:np.nan,-1:np.nan},
    'Number_of_days/week_walked_10+_minutes_Instance_0':{-3:np.nan,-2:np.nan,-1:np.nan},
    'Number_of_days/week_walked_10+_minutes_Instance_1':{-3:np.nan,-2:np.nan,-1:np.nan},
    'Number_of_days/week_walked_10+_minutes_Instance_2':{-3:np.nan,-2:np.nan,-1:np.nan},
    'Number_of_days/week_walked_10+_minutes_Instance_3':{-3:np.nan,-2:np.nan,-1:np.nan},
    'Ethnic_background_Instance_0':{-1:np.nan,-3:np.nan},
    'Ethnic_background_Instance_1':{-1:np.nan,-3:np.nan},
    'Ethnic_background_Instance_2':{-1:np.nan,-3:np.nan},
    'Cooked_vegetable_intake_Instance_0':{-3:np.nan,-1:np.nan,-10:0},
    'Cooked_vegetable_intake_Instance_1':{-3:np.nan,-1:np.nan,-10:0},
    'Cooked_vegetable_intake_Instance_2':{-3:np.nan,-1:np.nan,-10:0},
    'Cooked_vegetable_intake_Instance_3':{-3:np.nan,-1:np.nan,-10:0},
    'Salad_raw_vegetable_intake_Instance_0':{-3:np.nan,-1:np.nan,-10:0},
    'Salad_raw_vegetable_intake_Instance_1':{-3:np.nan,-1:np.nan,-10:0},
    'Salad_raw_vegetable_intake_Instance_2':{-3:np.nan,-1:np.nan,-10:0},
    'Salad_raw_vegetable_intake_Instance_3':{-3:np.nan,-1:np.nan,-10:0},
    'Fresh_fruit_intake_Instance_0':{-3:np.nan,-1:np.nan,-10:0},
    'Fresh_fruit_intake_Instance_1':{-3:np.nan,-1:np.nan,-10:0},
    'Fresh_fruit_intake_Instance_2':{-3:np.nan,-1:np.nan,-10:0},
    'Fresh_fruit_intake_Instance_3':{-3:np.nan,-1:np.nan,-10:0},
    'Processed_meat_intake_Instance_0':{-3:np.nan,-1:np.nan},
    'Processed_meat_intake_Instance_1':{-3:np.nan,-1:np.nan},
    'Processed_meat_intake_Instance_2':{-3:np.nan,-1:np.nan},
    'Processed_meat_intake_Instance_3':{-3:np.nan,-1:np.nan},
                     },inplace=True)

### Process Overall_acceleration_average based on two other field which indicate low data quality

In [ ]:
ashg_df_pandas['Overall_acceleration_average'][(ashg_df_pandas['Data_quality_good_calibration'] != 1) | (ashg_df_pandas['Data_quality_good_wear_time'] != 1)]=np.nan

In [ ]:
ashg_df_pandas.describe()

# 6. Process variables

## 6.1 Combine multiple-instance (multiple-measured) features

How to combine multiple measured features is a research question.

In [ ]:
walk=np.nanmean(np.array([ashg_df_pandas['Number_of_days/week_walked_10+_minutes_Instance_0']*ashg_df_pandas['Duration_of_walks_Instance_0'],
                    ashg_df_pandas['Number_of_days/week_walked_10+_minutes_Instance_1']*ashg_df_pandas['Duration_of_walks_Instance_1'],
                    ashg_df_pandas['Number_of_days/week_walked_10+_minutes_Instance_2']*ashg_df_pandas['Duration_of_walks_Instance_2'],
                    ashg_df_pandas['Number_of_days/week_walked_10+_minutes_Instance_3']*ashg_df_pandas['Duration_of_walks_Instance_3']
                   ]),axis=0)


In [ ]:
moderate=np.nanmean(np.array([ashg_df_pandas['Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_0']*ashg_df_pandas['Duration_of_moderate_activity_Instance_0'],
                              ashg_df_pandas['Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_1']*ashg_df_pandas['Duration_of_moderate_activity_Instance_1'],
                              ashg_df_pandas['Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_2']*ashg_df_pandas['Duration_of_moderate_activity_Instance_2'],
                              ashg_df_pandas['Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_3']*ashg_df_pandas['Duration_of_moderate_activity_Instance_3'],
                             ]),axis=0)



In [ ]:
vigorous=np.nanmean(np.array([ashg_df_pandas['Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_0']*ashg_df_pandas['Duration_of_vigorous_activity_Instance_0'],
                              ashg_df_pandas['Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_1']*ashg_df_pandas['Duration_of_vigorous_activity_Instance_1'],
                              ashg_df_pandas['Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_2']*ashg_df_pandas['Duration_of_vigorous_activity_Instance_2'],
                              ashg_df_pandas['Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_3']*ashg_df_pandas['Duration_of_vigorous_activity_Instance_3'],
                             ]),axis=0)


In [ ]:
bmi_average=ashg_df_pandas[['Body_mass_index_(BMI)_Instance_0','Body_mass_index_(BMI)_Instance_1','Body_mass_index_(BMI)_Instance_2','Body_mass_index_(BMI)_Instance_3']].mean(skipna=True,axis=1)

In [ ]:
def one_return(dataset):
    output=dataset.dropna().unique()
    if len(output)==1:
        return output[0]
    else:
        return np.nan

In [ ]:
ethnic=ashg_df_pandas[['Ethnic_background_Instance_0','Ethnic_background_Instance_1','Ethnic_background_Instance_2']].apply(one_return,axis=1)


In [ ]:
caucasian=ethnic.map(lambda x: 1 if (str(x).startswith('1')) else 0)
caucasian2=caucasian.astype('category')
caucasian2.value_counts()

In [ ]:
caucasian=caucasian2

In [ ]:
ethnic2=ethnic.astype('category')
ethnic2.value_counts()

In [ ]:
sleep=ashg_df_pandas[['Sleep_duration_Instance_0','Sleep_duration_Instance_1','Sleep_duration_Instance_2','Sleep_duration_Instance_3']].mean(skipna=True,axis=1)
alcohol=ashg_df_pandas[['Alcohol_intake_frequency._Instance_0','Alcohol_intake_frequency._Instance_1','Alcohol_intake_frequency._Instance_2','Alcohol_intake_frequency._Instance_3',]].mean(skipna=True,axis=1)
tv=ashg_df_pandas[['Time_spent_watching_television_(TV)_Instance_0','Time_spent_watching_television_(TV)_Instance_1','Time_spent_watching_television_(TV)_Instance_2','Time_spent_watching_television_(TV)_Instance_3',]].mean(skipna=True,axis=1)
computer=ashg_df_pandas[['Time_spent_using_computer_Instance_0','Time_spent_using_computer_Instance_1','Time_spent_using_computer_Instance_2','Time_spent_using_computer_Instance_3',]].mean(skipna=True,axis=1)
smoking=ashg_df_pandas[['Smoking_status_Instance_0','Smoking_status_Instance_1','Smoking_status_Instance_2','Smoking_status_Instance_3',]].mean(skipna=True,axis=1)
cholesterol=ashg_df_pandas[['Cholesterol_Instance_0','Cholesterol_Instance_1']].mean(skipna=True,axis=1)
HDL=ashg_df_pandas[['HDL_cholesterol_Instance_0','HDL_cholesterol_Instance_1']].mean(skipna=True,axis=1)
LDL=ashg_df_pandas[['LDL_direct_Instance_0','LDL_direct_Instance_1']].mean(skipna=True,axis=1)
triglyceride=ashg_df_pandas[['Triglycerides_Instance_0','Triglycerides_Instance_1']].mean(skipna=True,axis=1)
c_reactive=ashg_df_pandas[['Creactive_protein_Instance_0','Creactive_protein_Instance_1']].mean(skipna=True,axis=1)
HbA1c=ashg_df_pandas[['Glycated_haemoglobin_(HbA1c)_Instance_0','Glycated_haemoglobin_(HbA1c)_Instance_1']].mean(skipna=True,axis=1)
raw_vegie=ashg_df_pandas[['Salad_raw_vegetable_intake_Instance_0','Salad_raw_vegetable_intake_Instance_1','Salad_raw_vegetable_intake_Instance_2','Salad_raw_vegetable_intake_Instance_3']].mean(skipna=True,axis=1)
cook_vegie=ashg_df_pandas[['Cooked_vegetable_intake_Instance_0','Cooked_vegetable_intake_Instance_1','Cooked_vegetable_intake_Instance_2','Cooked_vegetable_intake_Instance_3']].mean(skipna=True,axis=1)
fresh_fruit=ashg_df_pandas[['Fresh_fruit_intake_Instance_0','Fresh_fruit_intake_Instance_1','Fresh_fruit_intake_Instance_2','Fresh_fruit_intake_Instance_3']].mean(skipna=True,axis=1)
process_meat=ashg_df_pandas[['Processed_meat_intake_Instance_0','Processed_meat_intake_Instance_1','Processed_meat_intake_Instance_2','Processed_meat_intake_Instance_3']].mean(skipna=True,axis=1)


In [ ]:
# Make reverse of normal alcohol to high number mean consuming more alcohol
alcohol_intake=6-alcohol

In [ ]:
alcohol.head()

In [ ]:
alcohol_intake.head()

## 6.2 Create label, bin variables, cast categorical variables

### Create label

In [ ]:
def create_HF_label(row):
    ICD_10_HF_list=['I500','I501','I509']
    ICD_9_HF_list=['4280','4281']
    ICD10=row.Diagnoses_ICD10
    ICD9=row.Diagnoses_ICD9
    if not ICD10 and not ICD9:
        # I encode no disease as missing data historically, but change to 0 after found that people with missing data is healthier (Data-Field 2178)
        # There is 'None', but no empty list in original dataset
        return 0
    if ICD10:
        if any(item in ICD10 for item in ICD_10_HF_list):
            return 1
    if ICD9:
        if any(item in ICD9 for item in ICD_9_HF_list):
            return 1

    return 0
        
    

In [ ]:
label=ashg_df_pandas.apply(create_HF_label, axis='columns')

### Bin variables

In [ ]:
def convert_townsend(value):
    if value <=-3.97:
        return 0
    elif value <=-2.84:
        return 1
    elif value <=-1.44:
        return 2
    elif value <=1.11:
        return 3
    elif value >1.11:
        return 4
    else:
        return np.nan

In [ ]:
def convert_acceleration(value):
    if value <=25:
        return 0
    elif value <=100:
        return 1
    elif value <=425:
        return 2
    elif value >425:
        return 3
    else:
        return np.nan

In [ ]:
townsend_grade=ashg_df_pandas['Townsend_deprivation_index_at_recruitment'].map(convert_townsend)
acceleration_grade=ashg_df_pandas['Overall_acceleration_average'].map(convert_acceleration)


In [ ]:
ashg_df_pandas['label']=label
ashg_df_pandas['townsend_grade']=townsend_grade
ashg_df_pandas['acceleration_grade']=acceleration_grade
ashg_df_pandas['label']=ashg_df_pandas['label'].astype('category')
ashg_df_pandas['Sex']=ashg_df_pandas['Sex'].astype('category')
ashg_df_pandas.drop(columns=['Diagnoses_ICD10','Diagnoses_ICD9','eid'],inplace=True)
ashg_df_pandas['caucasian']=caucasian

In [ ]:
def convert_bmi(value):
    if value <=25:
        return 0
    elif value <=30:
        return 1
    elif value <=35:
        return 2
    elif value >35:
        return 3
    else:
        return np.nan

In [ ]:
ashg_df_pandas['walk']=walk
ashg_df_pandas['moderate']=moderate
ashg_df_pandas['vigorous']=vigorous
ashg_df_pandas['bmi_grade']=bmi_average.map(convert_bmi)
ashg_df_pandas['bmi_average']=bmi_average
ashg_df_pandas['sleep']=sleep
ashg_df_pandas['alcohol']=alcohol_intake
ashg_df_pandas['tv']=tv
ashg_df_pandas['computer']=computer
ashg_df_pandas['smoking']=smoking
ashg_df_pandas['cholesterol']=cholesterol
ashg_df_pandas['HDL']=HDL
ashg_df_pandas['LDL']=LDL
ashg_df_pandas['triglyceride']=triglyceride
ashg_df_pandas['c_reactive']=c_reactive
ashg_df_pandas['HbA1c']=HbA1c
ashg_df_pandas['raw_vegie']=raw_vegie
ashg_df_pandas['cook_vegie']=cook_vegie
ashg_df_pandas['fresh_fruit']=fresh_fruit
ashg_df_pandas['process_meat']=process_meat



In [ ]:
ashg_df_pandas['bmi_grade'].value_counts()

In [ ]:
ashg_df_pandas.describe(include='all')

## 6.3 Remove individual with rarely missing values

In [ ]:
hf_df_processed=ashg_df_pandas.dropna(subset=['Sex','Age_at_recruitment'])

## 6.4 Drop unused fields

In [ ]:
hf_df_processed=hf_df_processed.drop(columns=[
                           'Body_mass_index_(BMI)_Instance_0','Body_mass_index_(BMI)_Instance_1','Body_mass_index_(BMI)_Instance_2','Body_mass_index_(BMI)_Instance_3',
                           'Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_0','Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_1',
                           'Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_2','Number_of_days/week_of_vigorous_physical_activity_10+_minutes_Instance_3',
                           'Duration_of_vigorous_activity_Instance_0','Duration_of_vigorous_activity_Instance_1','Duration_of_vigorous_activity_Instance_2','Duration_of_vigorous_activity_Instance_3',
                           'Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_0','Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_1',
                           'Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_2','Number_of_days/week_of_moderate_physical_activity_10+_minutes_Instance_3',
                           'Duration_of_moderate_activity_Instance_0','Duration_of_moderate_activity_Instance_1','Duration_of_moderate_activity_Instance_2','Duration_of_moderate_activity_Instance_3',
                           'Number_of_days/week_walked_10+_minutes_Instance_0','Number_of_days/week_walked_10+_minutes_Instance_1','Number_of_days/week_walked_10+_minutes_Instance_2','Number_of_days/week_walked_10+_minutes_Instance_3',
                           'Duration_of_walks_Instance_0','Duration_of_walks_Instance_1','Duration_of_walks_Instance_2','Duration_of_walks_Instance_3',
                           'Sleep_duration_Instance_0','Sleep_duration_Instance_1','Sleep_duration_Instance_2','Sleep_duration_Instance_3',
                           'Alcohol_intake_frequency._Instance_0','Alcohol_intake_frequency._Instance_1','Alcohol_intake_frequency._Instance_2','Alcohol_intake_frequency._Instance_3',
                           'Time_spent_watching_television_(TV)_Instance_0','Time_spent_watching_television_(TV)_Instance_1','Time_spent_watching_television_(TV)_Instance_2','Time_spent_watching_television_(TV)_Instance_3',
                           'Time_spent_using_computer_Instance_0','Time_spent_using_computer_Instance_1','Time_spent_using_computer_Instance_2','Time_spent_using_computer_Instance_3',
                           'Smoking_status_Instance_0','Smoking_status_Instance_1','Smoking_status_Instance_2','Smoking_status_Instance_3',
                           'Cholesterol_Instance_0','Cholesterol_Instance_1','Data_quality_good_calibration','Data_quality_good_wear_time',
                           'HDL_cholesterol_Instance_0','HDL_cholesterol_Instance_1',
                           'LDL_direct_Instance_0','LDL_direct_Instance_1',
                           'Triglycerides_Instance_0','Triglycerides_Instance_1',
                           'Creactive_protein_Instance_0','Creactive_protein_Instance_1',
                           'Glycated_haemoglobin_(HbA1c)_Instance_0','Glycated_haemoglobin_(HbA1c)_Instance_1',
                           'Salad_raw_vegetable_intake_Instance_0','Salad_raw_vegetable_intake_Instance_1','Salad_raw_vegetable_intake_Instance_2','Salad_raw_vegetable_intake_Instance_3',
                           'Cooked_vegetable_intake_Instance_0','Cooked_vegetable_intake_Instance_1','Cooked_vegetable_intake_Instance_2','Cooked_vegetable_intake_Instance_3',
                           'Fresh_fruit_intake_Instance_0','Fresh_fruit_intake_Instance_1','Fresh_fruit_intake_Instance_2','Fresh_fruit_intake_Instance_3',
                           'Processed_meat_intake_Instance_0','Processed_meat_intake_Instance_1','Processed_meat_intake_Instance_2','Processed_meat_intake_Instance_3',
                           'Ethnic_background_Instance_0','Ethnic_background_Instance_1','Ethnic_background_Instance_2'
                          ],inplace=False)


In [ ]:
hf_df_processed.describe(include='all')

In [ ]:
missing_value_count=hf_df_processed.isna().sum()

In [ ]:
missing_value_count.sort_values()

# 7. Univariate analyses

Inspect effect of individual feature

### Overall summary statistics

In [ ]:
hf_df_processed.columns

In [ ]:
hf_df_processed.groupby('label').agg(['median','mean'])

### Check self-reported PA

In [ ]:
hf_df_processed.groupby('label')[['walk','moderate','vigorous']].median()

In [ ]:
hf_df_processed.groupby('label')[['walk','moderate','vigorous']].mean()

In [ ]:
ax = sns.boxplot(x="label", y="walk",
                 data=hf_df_processed)
ax.set(ylim=(0, 1000))

In [ ]:
ax = sns.boxplot(x="label", y="moderate",
                 data=hf_df_processed)
ax.set(ylim=(0, 1000))

In [ ]:
ax = sns.boxplot(x="label", y="vigorous",
                 data=hf_df_processed)
ax.set(ylim=(0, 500))

### Check mean acceleration

In [ ]:
hf_df_processed.groupby('label')['Overall_acceleration_average'].median()

In [ ]:
ax = sns.boxplot(x="label", y="Overall_acceleration_average",
                 data=hf_df_processed)
ax.set(ylim=(0, 100))

### Age

In [ ]:
hf_df_processed.groupby('label')[['Age_at_recruitment']].mean()

In [ ]:
ax = sns.boxplot(x="label", y="Age_at_recruitment",
                 data=hf_df_processed)
ax.set(ylim=(0, 100))

### Sex

In [ ]:
hf_df_processed.groupby('label')['Sex'].value_counts()

In [ ]:
gender_count=hf_df_processed.groupby('label')['Sex'].value_counts()
sum_noHF=sum(gender_count[0])
sum_HF=sum(gender_count[1])
normalize_HF_by_grade=gender_count[1]/sum_HF
normalize_noHF_by_grade=gender_count[0]/sum_noHF
normalize_noHF_by_grade.sort_index(inplace=True)
normalize_HF_by_grade.sort_index(inplace=True)
fig, ax = plt.subplots()
width = 0.35  # the width of the bars
labels=['Female','Male']
positions=[0,1]
x=np.arange(len(labels))
rects1 = ax.bar(x - width/2, normalize_noHF_by_grade, width, label='noHF')
rects2 = ax.bar(x + width/2, normalize_HF_by_grade, width, label='HF')
#ax.bar(normalize_noHF_by_grade)
#ax.bar(normalize_HF_by_grade)
ax.legend()
plt.xticks(positions, labels)
plt.ylabel('Ratio')
plt.xlabel('Gender')

plt.show()

### BMI

In [ ]:
hf_df_processed.groupby('label')['bmi_average'].mean()

In [ ]:
hf_df_processed.groupby('label')[['bmi_average']].mean()

In [ ]:
ax = sns.boxplot(x="label", y="bmi_average",
                 data=hf_df_processed)

### Townsend deprivation

In [ ]:
ax = sns.boxplot(x="label", y="Townsend_deprivation_index_at_recruitment",
                 data=hf_df_processed)
#ax.set(ylim=(0, 10))

### Cholesterol

In [ ]:
ax = sns.boxplot(x="label", y="cholesterol",
                 data=hf_df_processed)
ax.set(ylim=(0, 10))

### TV

In [ ]:
ax = sns.boxplot(x="label", y="tv",
                 data=hf_df_processed)
ax.set(ylim=(0, 10))

### Computer

In [ ]:
ax = sns.boxplot(x="label", y="computer",
                 data=hf_df_processed)


### Walk

In [ ]:
ax = sns.boxplot(x="label", y="walk",
                 data=hf_df_processed)
#ax.set(ylim=(0, 10))

### Sleep

In [ ]:
ax = sns.boxplot(x="label", y="sleep",
                 data=hf_df_processed)


### Alcohol

In [ ]:
ax = sns.boxplot(x="label", y="alcohol",
                 data=hf_df_processed)
ax.set(ylim=(0, 10))

### Smoking

In [ ]:
ax = sns.boxplot(x="label", y="smoking",
                 data=hf_df_processed)


### Blood test

In [ ]:
ax = sns.boxplot(x="label", y="HDL",
                 data=hf_df_processed)

In [ ]:
ax = sns.boxplot(x="label", y="LDL",
                 data=hf_df_processed)

In [ ]:
ax = sns.boxplot(x="label", y="triglyceride",
                 data=hf_df_processed)

In [ ]:
ax = sns.boxplot(x="label", y="c_reactive",
                 data=hf_df_processed)

In [ ]:
ax = sns.boxplot(x="label", y="HbA1c",
                 data=hf_df_processed)

### Food

In [ ]:
ax = sns.boxplot(x="label", y="raw_vegie",
                 data=hf_df_processed)

In [ ]:
ax = sns.boxplot(x="label", y="cook_vegie",
                 data=hf_df_processed)

In [ ]:
ax = sns.boxplot(x="label", y="fresh_fruit",
                 data=hf_df_processed)

In [ ]:
ax = sns.boxplot(x="label", y="process_meat",
                 data=hf_df_processed)

In [ ]:
hf_df_processed=hf_df_processed.drop(columns=[
    'townsend_grade','acceleration_grade','bmi_grade'
                          ],inplace=False)


In [ ]:
missing_value_count=hf_df_processed.isna().sum().sort_values()

In [ ]:
100*missing_value_count.sort_values()/hf_df_processed.shape[0]


#### We gonna remove Overall_acceleration_average later for multivariate or ML analysis due to its high missing value. 

### Save to csv for R and other stat analyhsis

In [ ]:
hf_df_processed.to_csv('hf_df_processed.csv',na_rep='NA',index=False)

In [ ]:
! dx upload hf_df_processed.csv  --path /ASHGdemo/hf_df_processed.csv

### Once again, we may start from hf_df_processed.csv going foward. 

#### here is how to load data as CSV

`hf_df_processed = pd.read_csv('hf_df_processed.csv', sep=',')`

#### check datatype

`hf_df_processed.info()`

#### change to use categorical

`hf_df_processed['label']=hf_df_processed['label'].astype('category')`

`hf_df_processed['Sex']=hf_df_processed['Sex'].astype('category')`

`hf_df_processed['caucasian']=hf_df_processed['caucasian'].astype('category')`

#### check datatype again

`hf_df_processed.info()`


# 8. Basic machine learning

### Install all modules we will use and import them

In [ ]:
pip install seaborn==0.11.1

In [ ]:
! pip install -U imbalanced-learn

In [ ]:
! pip install sklearn

In [ ]:
pip install xgboost

In [ ]:
pip install lightgbm

In [ ]:
pip install shap

In [ ]:
! pip install interpret==0.2.6

## restart kernal here and import all the package we need

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedRandomForestClassifier

In [ ]:
from IPython.display import Image

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import json
from collections import Counter


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, log_loss


In [ ]:
from xgboost import XGBClassifier


In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from interpret.provider import InlineProvider
from interpret import set_visualize_provider

set_visualize_provider(InlineProvider())

In [ ]:
from interpret import show
from interpret.data import ClassHistogram

In [ ]:
from interpret import set_show_addr, get_show_addr 
from interpret.glassbox import ExplainableBoostingClassifier


## Load data from CSV

In [ ]:
! dx download /ASHGdemo/hf_df_processed.csv

In [ ]:
hf_df_processed = pd.read_csv('hf_df_processed.csv', sep=',')
hf_df_processed['label']=hf_df_processed['label'].astype('category')
hf_df_processed['Sex']=hf_df_processed['Sex'].astype('category')
hf_df_processed['caucasian']=hf_df_processed['caucasian'].astype('category')

In [ ]:
hf_df_processed.shape

In [ ]:
hf_df_processed.dropna(subset=['label'],inplace=True)

In [ ]:
hf_df_processed.shape

In [ ]:
hf_df_processed.info()

In [ ]:
hf_df_processed.isna().sum().sort_values()

In [ ]:
hf_df_processed.isna().sum().sort_values()*100/(hf_df_processed.shape[0])

## Remove feature with high missing value

In [ ]:
hf_df_good_measure=hf_df_processed.drop(columns=['Overall_acceleration_average'])

## Separate between full data set and dataset with no missing value(Complete Case Analysis)

In [ ]:
hf_df_good_measure.label.value_counts()

In [ ]:
CCA_hf_df_good_measure=hf_df_good_measure.dropna()

In [ ]:
CCA_hf_df_good_measure.shape

In [ ]:
CCA_hf_df_good_measure.label.value_counts()

## Use downsampling to balance class and separate out label

In [ ]:
X = hf_df_good_measure.drop(columns=['label'])
y = hf_df_good_measure['label']

In [ ]:
counter = Counter(y)
estimate = counter[0] / counter[1]
estimate

In [ ]:
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X, y)

In [ ]:
y_resampled.value_counts()

In [ ]:
CCA_X = CCA_hf_df_good_measure.drop(columns=['label'])
CCA_y = CCA_hf_df_good_measure['label']
counter = Counter(CCA_y)
CCA_estimate = counter[0] / counter[1]   
CCA_estimate

In [ ]:
rus = RandomUnderSampler(random_state=0)
CCA_X_resampled, CCA_y_resampled = rus.fit_resample(CCA_X, CCA_y)

In [ ]:
CCA_y_resampled.value_counts()

In [ ]:
## One hot encoding categorical 

In [ ]:
object_cols=['Sex','caucasian']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_resampled , y_resampled , train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [ ]:
CCA_X_train, CCA_X_valid, CCA_y_train, CCA_y_valid = train_test_split(CCA_X_resampled , CCA_y_resampled , train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(CCA_X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(CCA_X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = CCA_X_train.index
OH_cols_valid.index = CCA_X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = CCA_X_train.drop(object_cols, axis=1)
num_X_valid = CCA_X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_CCA_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_CCA_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

## Run multiple ML prediction model

Run models in the following order:

    * XGboosted on full dataset
    * XGboosted on CCA (no missing value)
    * lightGBM on full dataset
    * lightGBM on CCA
    * random forest on CCA
    * logistic regression on CCA
    * Explaininable boosting machine (EBM) on CCA

Note that random forest, logistic regression, and EBM can't handle missing values, so we only use them on CCA.
    

In [ ]:
xgb_full= XGBClassifier(n_estimators=1000)
xgb_full.fit(OH_X_train,y_train,early_stopping_rounds=5,
            eval_set=[(OH_X_valid, y_valid)],eval_metric=['auc','logloss'])

In [ ]:
xgb_CCA= XGBClassifier(n_estimators=1000)
xgb_CCA.fit(OH_CCA_X_train,CCA_y_train,early_stopping_rounds=5,
            eval_set=[(OH_CCA_X_valid, CCA_y_valid)],eval_metric=['auc','logloss'])

In [ ]:
lgbm_full=LGBMClassifier(n_estimators=1000)
lgbm_full.fit(OH_X_train,y_train,early_stopping_rounds=5,
            eval_set=[(OH_X_valid, y_valid)],eval_metric=['auc','logloss'])

In [ ]:
lgbm_CCA=LGBMClassifier(n_estimators=1000)
lgbm_CCA.fit(OH_CCA_X_train,CCA_y_train,early_stopping_rounds=5,
            eval_set=[(OH_CCA_X_valid, CCA_y_valid)],eval_metric=['auc','logloss'])

In [ ]:
forest_CCA = RandomForestClassifier(n_estimators=100,random_state=0)
forest_CCA.fit(OH_CCA_X_train,CCA_y_train)

In [ ]:
logistic_CCA=LogisticRegression(random_state=0,solver='lbfgs',max_iter=10000)# solver='saga')
logistic_CCA.fit(OH_CCA_X_train,CCA_y_train)

In [ ]:
ebm_CCA = ExplainableBoostingClassifier(random_state=127,max_rounds=5000)
ebm_CCA.fit(CCA_X_train, CCA_y_train) 

## Use the following cells to analyze accuracy and F1 score in test and testing dataset for models that run on full dataset

In [ ]:
model_full=lgbm_full
preds_train = model_full.predict(OH_X_train)
preds_val = model_full.predict(OH_X_valid)


In [ ]:
accuracy_train=accuracy_score(y_train,preds_train)
accuracy_val=accuracy_score(y_valid,preds_val)
f1_train=f1_score(y_train,preds_train)
f1_val=f1_score(y_valid,preds_val)

print('predicted train {}'.format(Counter(preds_train)))
print('predicted valid {}'.format(Counter(preds_val)))
print('Train accuracy {} and F1 {}'.format(accuracy_train,f1_train))
print('Validation accuracy {} and F1 {}'.format(accuracy_val,f1_val))

## Use the following cells to analyze accuracy and F1 score in test and testing dataset for models that run on CCA

In [ ]:
model_CCA=lgbm_CCA
preds_train = model_CCA.predict(OH_CCA_X_train)
preds_val = model_CCA.predict(OH_CCA_X_valid)

In [ ]:
accuracy_train=accuracy_score(CCA_y_train,preds_train)
accuracy_val=accuracy_score(CCA_y_valid,preds_val)
f1_train=f1_score(CCA_y_train,preds_train)
f1_val=f1_score(CCA_y_valid,preds_val)

print('predicted train {}'.format(Counter(preds_train)))
print('predicted valid {}'.format(Counter(preds_val)))
print('Train accuracy {} and F1 {}'.format(accuracy_train,f1_train))
print('Validation accuracy {} and F1 {}'.format(accuracy_val,f1_val))

### The exception is EBM which we dont' need one hot encoding

In [ ]:
preds_train = ebm_CCA.predict(CCA_X_train)
preds_val = ebm_CCA.predict(CCA_X_valid)

### Here are results we get when make poster

# 9. Interpretable and explainable ML


In [ ]:
import shap

In [ ]:
print(shap.__version__)

### We will run SHAP on all model to investigate order of feature importance and their affects on each individual

In [ ]:
explainer_xgb_full = shap.TreeExplainer(xgb_full)
shap_values_xgb_full = explainer_xgb_full.shap_values(OH_X_valid,approximate=True)

In [ ]:
shap.summary_plot(shap_values_xgb_full, OH_X_valid,plot_type="bar")


In [ ]:
shap.summary_plot(shap_values_xgb_full, OH_X_valid)

In [ ]:
explainer_xgb_CCA = shap.TreeExplainer(xgb_CCA)
shap_values_xgb_CCA = explainer_xgb_CCA.shap_values(OH_CCA_X_train,approximate=True)
shap.summary_plot(shap_values_xgb_CCA, OH_CCA_X_train,plot_type="bar")

In [ ]:
shap.summary_plot(shap_values_xgb_CCA, OH_CCA_X_train)


In [ ]:
explainer_xgb_CCA2 = shap.TreeExplainer(xgb_CCA)
shap_values_xgb_CCA2 = explainer_xgb_CCA2.shap_values(OH_CCA_X_train)

In [ ]:
shap_values_xgb_CCA2.shape

In [ ]:
shap_values_xgb_CCA.shape

In [ ]:
explainer_lgbm_full = shap.TreeExplainer(lgbm_full)
shap_values_lgbm_full = explainer_lgbm_full.shap_values(OH_X_valid)
shap.summary_plot(shap_values_lgbm_full[1], OH_X_valid,plot_type="bar")

In [ ]:
shap.summary_plot(shap_values_lgbm_full[1], OH_X_valid)

#### The lbgm_full has the best performance on test dataset, so we will also investigate partial dependent plot to see overall trend and scale of impact.

In [ ]:
X100 = shap.utils.sample(OH_X_valid, 100)

In [ ]:
OH_X_valid.columns

In [ ]:
shap.plots.partial_dependence(
    "Age_at_recruitment", lgbm_full.predict, X100, ice=False,
    model_expected_value=True, feature_expected_value=True
)

In [ ]:
shap.plots.partial_dependence(
    "bmi_average", lgbm_full.predict, X100, ice=False,
    model_expected_value=True, feature_expected_value=True
)

In [ ]:
shap.plots.partial_dependence(
    "HbA1c", lgbm_full.predict, X100, ice=False,
    model_expected_value=True, feature_expected_value=True
)

In [ ]:
shap.plots.partial_dependence(
    "cholesterol", lgbm_full.predict, X100, ice=False,
    model_expected_value=True, feature_expected_value=True
)

In [ ]:
shap.plots.partial_dependence(
    -4, lgbm_full.predict, X100, ice=False,
    model_expected_value=True, feature_expected_value=True
)

Notice that it doesn't understand categorical feature and pick a different kind of plotting. Even though this is compresible, it would be better to use different visuzliation for categorical from numerical. We will address this later.

In [ ]:
explainer_lgbm_CCA = shap.TreeExplainer(lgbm_CCA)
shap_values_lgbm_CCA = explainer_lgbm_CCA.shap_values(OH_CCA_X_train)
shap.summary_plot(shap_values_lgbm_CCA[1], OH_CCA_X_train,plot_type="bar")


In [ ]:
shap.summary_plot(shap_values_lgbm_CCA[1], OH_CCA_X_train)


In [ ]:
print(len(shap_values_lgbm_CCA))
shap_values_lgbm_CCA[0].shape

In [ ]:
explainer_forest_CCA = shap.TreeExplainer(forest_CCA)
shap_values_forest_CCA = explainer_forest_CCA.shap_values(OH_CCA_X_valid,approximate=True)
shap.summary_plot(shap_values_forest_CCA[1], OH_CCA_X_valid,plot_type="bar")

In [ ]:
shap.summary_plot(shap_values_forest_CCA[1], OH_CCA_X_valid)

In [ ]:
explainer_logistic_CCA = shap.Explainer(logistic_CCA,OH_CCA_X_valid)
shap_values_logistic_CCA = explainer_logistic_CCA(OH_CCA_X_valid)
shap.summary_plot(shap_values_logistic_CCA, OH_CCA_X_valid,plot_type="bar")

In [ ]:
list(zip(list(logistic_CCA.coef_[0]),list(OH_CCA_X_valid.columns)))

One hot encoding for logis is hard to interpret. It is confusing which number represent what. Also, how come the pair number (0 vs 1 and 2 vs 3) have different value. We will try with non-one hot encoding instead.

In [ ]:
logistic_CCA_noOH=LogisticRegression(random_state=0,solver='lbfgs',max_iter=10000)# solver='saga')
logistic_CCA_noOH.fit(CCA_X_train,CCA_y_train)

In [ ]:
preds_train = logistic_CCA_noOH.predict(CCA_X_train)
preds_val = logistic_CCA_noOH.predict(CCA_X_valid)
accuracy_train=accuracy_score(CCA_y_train,preds_train)
accuracy_val=accuracy_score(CCA_y_valid,preds_val)
f1_train=f1_score(CCA_y_train,preds_train)
f1_val=f1_score(CCA_y_valid,preds_val)
print('predicted train {}'.format(Counter(preds_train)))
print('predicted valid {}'.format(Counter(preds_val)))
print('Train accuracy {} and F1 {}'.format(accuracy_train,f1_train))
print('Validation accuracy {} and F1 {}'.format(accuracy_val,f1_val))

In [ ]:
explainer_logistic_CCA_noOH = shap.Explainer(logistic_CCA_noOH,CCA_X_valid)
shap_values_logistic_CCA_noOH = explainer_logistic_CCA_noOH(CCA_X_valid)
shap.summary_plot(shap_values_logistic_CCA_noOH, CCA_X_valid,plot_type="bar")

This model has lower predictive power than its one-hot encoding correspondance.

### Finally, the EBM. Rather than running SHAP which is not current support, we will use build-in interpret feature to inspect global feature imporant and partial dependent plot instead

In [ ]:
ebm_CCA_global = ebm_CCA.explain_global(name='EBM')
show(ebm_CCA_global)

In the poster, we use SHAP analysis for most important features for each model (except EBM which has its own). All boosted tree model have similar predictive power and most important feature with lightGBM full has highest validation F1. However, we choose to show partial dependent plot and population density from EBM (package interpretML/interpret) since it's easier to understand. 

Thanks for reading this notebook. Hope this example here would be useful for your research.